# Terminal

This set of tools gives AI agents the ability to fully interact with a terminal window, including sending special
keystrokes like Escape or Ctrl+C, and seeing the terminal window's contents.

Highly recommended to use only inside a container or other isolated environment.

> Note: As of May 2024, the current generation of language models aren't able to effectively utilize the terminal tool,
> and get confused by the keystrokes they have to send, and the output they see.
> However, at the rate progress is being made, we can expect this limitation to be gone by the end of the year.
> Then the advantage of the power of the terminal and the myriad tools available for it will become apparent.

Terminal interaction depends on tmux.
Follow [these instructions](https://github.com/tmux/tmux/wiki/Installing) to install tmux on your Unix-like system.
If you're running Windows, you'll need to install Windows Subsystem for Linux (WSL) first.

After you have the tmux system package installed, install the `libtmux` Python package.

In [ ]:
%pip install --upgrade --quiet libtmux > /dev/null

Now you can test the tool out yourself:

In [8]:
import time

from langchain_community.tools import (
    TerminalBottomCaptureTool,
    TerminalLiteralInputTool,
    TerminalSpecialInputTool,
)

TerminalLiteralInputTool().run("echo hello world")
TerminalSpecialInputTool().run("Enter")
time.sleep(0.1)
print(TerminalBottomCaptureTool().run("3"))

vscode ➜ .../docs/docs/integrations/tools (terminal-window-tool) $ echo hello world
hello world
vscode ➜ .../docs/docs/integrations/tools (terminal-window-tool) $


Note that you can run `tmux attach` to attach to your agent's session and see what your agent is doing in real time.

You can also do more sophisticated things like use Vim.
Install Vim first if you want to run the example yourself.

In [10]:
import time

from langchain_community.tools import (
    TerminalBottomCaptureTool,
    TerminalLiteralInputTool,
    TerminalSpecialInputTool,
)

TerminalLiteralInputTool().run("vim /tmp/test.txt")
TerminalSpecialInputTool().run("Enter")
time.sleep(0.5)
TerminalLiteralInputTool().run("i")
time.sleep(0.5)
print(TerminalBottomCaptureTool().run("1"))
TerminalLiteralInputTool().run("This is a line of text")
TerminalSpecialInputTool().run("Escape")
TerminalLiteralInputTool().run(":w")
TerminalSpecialInputTool().run("Enter")
TerminalSpecialInputTool().run(
    "^Z"
)  # Ctrl+Z to exit abruptly, will cause the swap file to be retained.
TerminalLiteralInputTool().run("cat /tmp/test.txt")
TerminalSpecialInputTool().run("Enter")
print(TerminalBottomCaptureTool().run("6"))

-- INSERT --                                                                                                              0,1           All
vscode ➜ .../docs/docs/integrations/tools (terminal-window-tool) $ vim /tmp/test.txt

[13]+  Stopped                 vim /tmp/test.txt
vscode ➜ .../docs/docs/integrations/tools (terminal-window-tool) $ cat /tmp/test.txt
This is a line of text
vscode ➜ .../docs/docs/integrations/tools (terminal-window-tool) $


You can let your LM use the tools too. Example using OpenAI below.
You'll need to run `pip install langchain-openai` first if you haven't already.

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [4]:
import time

from langchain_community.tools import (
    TerminalBottomCaptureTool,
    TerminalLiteralInputTool,
    TerminalSpecialInputTool,
)
from langchain_openai import ChatOpenAI

lm = ChatOpenAI(model="gpt-4o-2024-05-13").bind_tools(
    [
        TerminalLiteralInputTool(),
        TerminalSpecialInputTool(),
        TerminalBottomCaptureTool(),
    ]
)

msg = lm.invoke("What top 3 processes consuming the most memory?")
msg.tool_calls

[{'name': 'terminal_literal_input',
  'args': {'__arg1': 'ps aux --sort=-%mem | head -n 4'},
  'id': 'call_xS2CDWlgZFkslqe7lM2QBoBz'},
 {'name': 'terminal_special_input',
  'args': {'__arg1': 'Enter'},
  'id': 'call_p6b4MVlPZ5FdC2aWsk2F4dEo'}]